# W207, Final Project
Spring, 2018

Team:  Cameron Kennedy, Gaurav Khanna, Aaron Olson

## Data Preparation / Feature Extraction Notebook
Python Notebook 1 of 2

This notebook loads and pre-processes the data.  The other notebook (2 of 2) runs our ML models.

# Introduction
This analysis seeks to predict user churn in a music sharing service.

We will write a more complete description and analysis for submission of our final project.

We worked on 2 major data tables/frames (User logs & Transactions) independently for preperation and then brought them together before analysis

In [ ]:
#Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Loading the data indexing with the primary key (MSNO: String like/Object, represents the user)

In [2]:
#Load the data
members = pd.read_csv('members_filtered.csv')
transactions = pd.read_csv('transactions_filtered.csv')
user_logs = pd.read_csv('user_logs_filtered.csv')
labels = pd.read_csv('labels_filtered.csv')

#Set indices
members.set_index('msno', inplace = True)
labels.set_index('msno', inplace = True)

#user_logs.head()

Getting some info on the userful data

In [3]:

print('Transactions: \n')
transactions.info()
print('User Logs: \n')
user_logs.info()

Transactions: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282160 entries, 0 to 282159
Data columns (total 9 columns):
msno                      282160 non-null object
payment_method_id         282160 non-null int64
payment_plan_days         282160 non-null int64
plan_list_price           282160 non-null int64
actual_amount_paid        282160 non-null int64
is_auto_renew             282160 non-null int64
transaction_date          282160 non-null int64
membership_expire_date    282160 non-null int64
is_cancel                 282160 non-null int64
dtypes: int64(8), object(1)
memory usage: 19.4+ MB
User Logs: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883573 entries, 0 to 4883572
Data columns (total 9 columns):
msno          object
date          int64
num_25        int64
num_50        int64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(1), int64(7), object(1)
memory usage: 335.3+ MB


Helper routine to format the date for visualization. Not conducive for analysis though

In [5]:
def pd_to_date(df_col):
    df_col = pd.to_datetime(df_col, format = '%Y%m%d')
    return df_col

#Convert to date
user_logs['date'] = pd_to_date(user_logs['date'])
#user_logs.head()

# User Logs Data: Preparation and Feature Extraction

In [6]:
#Create our groupby user object 
user_logs_gb = user_logs.groupby(['msno'], sort=False)

In [7]:
#This cell is slow

#Append max date to every row in main table
user_logs['max_date'] = user_logs_gb['date'].transform('max')
user_logs['days_before_max_date'] = (user_logs['max_date'] - user_logs['date']).apply(lambda x: x.days)
    #The .apply(lambda...  just converts it from datetime to an integer, for easier comparisons later.

#Generate user's first date, last date, and tenure
#Also, the user_logs_features table will be the primary table to return from the transactions table
user_logs_features = (user_logs_gb
    .agg({'date':['max', 'min', lambda x: (max(x) - min(x)).days]})  #.days converts to int
    .rename(columns={'max': 'max_date', 'min': 'min_date','<lambda>':'listening_tenure'})
                      )
#Add a 3rd level, used for joining data later
user_logs_features = pd.concat([user_logs_features], axis=1, keys=['date_features'])

In [8]:
user_logs_features.head()

date_features             \
                                                      date              
                                                  max_date   min_date   
msno                                                                    
AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=    2017-02-24 2016-01-25   
f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=    2017-02-28 2015-01-02   
Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=    2017-02-28 2016-02-07   
Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=    2017-02-24 2015-01-02   
mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=    2017-02-28 2015-12-20   

                                                               
                                                               
                                             listening_tenure  
msno                                                           
AtRE9/xHjNrX8tcuUb/cdBRvUHpwXhZrWY9E9sk1tDk=              396  
f3HbYjEEVBVeWcP3FNDCLKJXpuIZPvX0oFyn4XLMN3k=              788  
Q7nR6DbodHwUTGPi5Z0yOsdC2cSOhiWIe4cNEVmnJ9I=              387  
Qx9cihonlt2hkBa3VHNj+nwX2QelZXpyHnEAIjD199w=              784  
mf3zFfyeZLl8zEWhV+GyWQYjt3orfQUCcOIMwBJ60+k=              436

In [9]:
#Create Features:
    # Total X=(seconds, 100, 985, 75, 50, 25, unique), avg per day of X, maybe median per day of X
    # Last day, last 7 days, last 30 days, last 90, 180, 365, total (note last day is relative to user)
    
for num_days in [7, 14, 31, 90, 180, 365, 999]:
    #Create groupby object for items with x days
    ul_gb_xdays = (user_logs.loc[(user_logs['days_before_max_date'] < num_days)]
                   .groupby(['msno'], sort=False))

    #Generate sum and mean (and count, once) for all the user logs stats
    past_xdays_by_user = (ul_gb_xdays
        .agg({'num_unq':['sum', 'mean', 'count'],
              'total_secs':['sum', 'mean'],
              'num_25':['sum', 'mean'],
              'num_50':['sum', 'mean'],
              'num_75':['sum', 'mean'],
              'num_985':['sum', 'mean'],
              'num_100':['sum', 'mean'],
             })
                      )
    #Append level header
    past_xdays_by_user = pd.concat([past_xdays_by_user], axis=1, keys=['within_days_' + str(num_days)])

    #Join (append) to user_logs_features table
    user_logs_features = user_logs_features.join(past_xdays_by_user, how='inner')

In [ ]:
#Next, let's look at changes in last 7 days vs. last 30 days, and last 30 days vs. last 180 days.

#Also, need to think about users with < x days tenure.

In [10]:
#Join members and labels files
features_all = None
features_all = members.join(labels, how='inner')
features_all = features_all.join(user_logs_features, how='inner')

#Note, the warning is okay, and actually helps us by flattening our column headers.

# Test
features_all.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


,city,bd,gender,registerd_via,registration_init_time,is_churn,"(date_features, date, max_date)","(date_features, date, min_date)","(date_features, date, listening_tenure)","(within_days_7, num_unq, sum)",...,"(within_days_999, num_25, sum)","(within_days_999, num_25, mean)","(within_days_999, num_50, sum)","(within_days_999, num_50, mean)","(within_days_999, num_75, sum)","(within_days_999, num_75, mean)","(within_days_999, num_985, sum)","(within_days_999, num_985, mean)","(within_days_999, num_100, sum)","(within_days_999, num_100, mean)"
msno,,,,,,,,,,,,,,,,,,,,,
SlQEFsP8unoHeCV5naZtlhd9klqGJyTkYXaudM1ls1s=,1,0,NaN,13,20160918,0,2017-02-28,2016-09-20,161,18,...,332,2.886957,182,1.582609,99,0.860870,151,1.313043,2787,24.234783
0XXpgJ9L7btDN0H/y+R5sRwIFDxDiRoRPU1U/jh1f9M=,1,0,NaN,13,20161224,0,2017-02-28,2017-01-16,43,35,...,170,11.333333,40,2.666667,8,0.533333,4,0.266667,87,5.800000
JAyCmh9aHTsNe79eT0qtdU7VubhYitygdTdU4T3fZlA=,1,0,NaN,13,20170121,0,2017-02-28,2017-01-21,38,20,...,221,9.208333,55,2.291667,16,0.666667,11,0.458333,279,11.625000
XzwNNw5S1qNCNjjF23l6DVe8vuWRPFmIy6bmnZVq8Qo=,1,0,NaN,13,20161004,0,2017-02-28,2016-10-04,147,130,...,1146,8.426471,150,1.102941,127,0.933824,113,0.830882,5182,38.102941
zXjQ9f4C/uJHWw5guWICNbkFBydqdrz77Apch1nyDQ4=,1,0,NaN,13,20170108,0,2017-02-28,2017-01-09,50,16,...,17,0.809524,15,0.714286,7,0.333333,3,0.142857,246,11.714286


# Transaction Data: Preparation and Feature Extraction

Grouping by the primary key (MSNO)

In [11]:
# Grouping by the member (msno)
transactions_gb = transactions.sort_values(["transaction_date"]).groupby(['msno'])

# How many groups i.e. members i.e. msno's. We're good if this is the same number as the members table
print('%d Groups/msnos' %(len(transactions_gb.groups)))

17375 Groups/msnos


The list of features 

    * Latest transaction
        * Plan no of days for the latest transaction
        * Plan actual amount paid/day for the latest transaction
        * plan total amount paid for the latest transaction
        * Is_auto_renew for the latest transaction
        * is_cancel for the latest transaction
    * Aggregate values
        * Total number of plan days
        * Total of all the amounts paid for the plan
    * Comparing transactions
        * Plan day difference among the latest and previous transaction
        * Amount paid/day difference among the latest and previous transaction
    ....


Aggregate values

In [12]:
# Features: Total_plan_days, Total_amount_paid
transactions_features = (transactions_gb
    .agg({'payment_plan_days':'sum', 'actual_amount_paid':'sum' })
    .rename(columns={'payment_plan_days': 'Total_plan_days', 'actual_amount_paid': 'Total_amount_paid',})
          )
# Test
# transactions_features.head()

Latest transaction. We'll just pick from the bottom of the ordered (by date) rows in groups

In [13]:
# Features: latest transaction, renaming the collumns
latest_transaction_gb = transactions_gb.tail([1]).rename(columns={'payment_plan_days': 'latest_plan_days', 'actual_amount_paid': 'latest_amount_paid','is_auto_renew': 'latest_auto_renew', 
                                                                  'transaction_date': 'latest_transaction_date',
                                                                  'membership_expire_date': 'latest_expire_date', 'is_cancel': 'latest_is_cancel' })

# Index by msno
latest_transaction_gb.set_index('msno', inplace = True)

# Test
# latest_transaction_gb.head()

In [14]:
# Plan actual amount paid/day for the latest transaction
# Adding the collumn amount_paid_per_day

latest_transaction_gb['amount_paid_per_day'] = (latest_transaction_gb['latest_amount_paid']/latest_transaction_gb['latest_plan_days'])

# Test
latest_transaction_gb.head()

,payment_method_id,latest_plan_days,plan_list_price,latest_amount_paid,latest_auto_renew,latest_transaction_date,latest_expire_date,latest_is_cancel,amount_paid_per_day
msno,,,,,,,,,
NRzJYmVfPD9TfCGGP9ai1fHAr69INjKXREgt6ViaYwA=,38,450,1788,1788,0,20150630,20170218,0,3.973333
GVfMVTbKtQgHkmTaJZq0PSM/49neJxN2CdM/j6SGGNs=,41,30,149,149,1,20150919,20170320,0,4.966667
NSHsvkXPX2yjnIzWXW373xGMssTh1u26rqRVzHadXR0=,32,410,1788,1788,0,20150923,20170203,0,4.360976
H3036d7cObAUujRSPEq7xEHJboHgQJ58Kv0zWfJKh0E=,22,395,1599,1599,0,20151013,20170209,0,4.048101
tT2aF3w8Ox+2bC2GaEU/jp8dVvhhAO5inykLclBtXy8=,32,395,1599,1599,0,20151112,20170213,0,4.048101


In [15]:
# TODO Differences among latest and previous transaction

Getting all the transaction features in a single DF

In [16]:
# Get all transaction features in a single DF
transactions_features = transactions_features.join(latest_transaction_gb, how = 'inner')

# Test
transactions_features.head()

,Total_plan_days,Total_amount_paid,payment_method_id,latest_plan_days,plan_list_price,latest_amount_paid,latest_auto_renew,latest_transaction_date,latest_expire_date,latest_is_cancel,amount_paid_per_day
msno,,,,,,,,,,,
++L+G2jsvbkHMHlwvb2KQiRLAvB4VyEfjVJTUrs5auE=,720,3725,38,30,149,149,0,20170216,20170318,0,4.966667
++XZCubnx3mWCWwaVKNyjfZjYZTysD8qvfPb/QMomeI=,240,1192,41,30,149,149,1,20170106,20170107,1,4.966667
++am6f+rLDE3gjQM7pKLVAthwCgaI46WHbTNuKtgpbI=,780,3354,41,30,129,129,1,20170214,20170315,0,4.300000
++k5broOoWP/P2WkW2N4C/sXL2bowW56Ep/emCCafeQ=,210,1043,39,30,149,149,1,20170131,20170311,0,4.966667
++kqM73xL/v0vqbSItFKWgtEyIkW2POP4c/SEA0WZmw=,180,894,41,30,149,149,1,20170123,20170123,1,4.966667


# Bringing all the features in a single Data Frame, file

Members and Labels were joined into the User logs DF
We're joining the Transaction and User log DF below

In [18]:
# Joining feature DF's
features_all = features_all.join(transactions_features, how='inner')

In [19]:
# Test
features_all.head()

,city,bd,gender,registerd_via,registration_init_time,is_churn,"(date_features, date, max_date)","(date_features, date, min_date)","(date_features, date, listening_tenure)","(within_days_7, num_unq, sum)",...,Total_amount_paid,payment_method_id,latest_plan_days,plan_list_price,latest_amount_paid,latest_auto_renew,latest_transaction_date,latest_expire_date,latest_is_cancel,amount_paid_per_day
msno,,,,,,,,,,,,,,,,,,,,,
SlQEFsP8unoHeCV5naZtlhd9klqGJyTkYXaudM1ls1s=,1,0,NaN,13,20160918,0,2017-02-28,2016-09-20,161,18,...,745,30,30,149,149,1,20170219,20170318,0,4.966667
0XXpgJ9L7btDN0H/y+R5sRwIFDxDiRoRPU1U/jh1f9M=,1,0,NaN,13,20161224,0,2017-02-28,2017-01-16,43,35,...,387,30,30,129,129,1,20170224,20170323,0,4.300000
JAyCmh9aHTsNe79eT0qtdU7VubhYitygdTdU4T3fZlA=,1,0,NaN,13,20170121,0,2017-02-28,2017-01-21,38,20,...,200,30,30,100,100,1,20170222,20170321,0,3.333333
XzwNNw5S1qNCNjjF23l6DVe8vuWRPFmIy6bmnZVq8Qo=,1,0,NaN,13,20161004,0,2017-02-28,2016-10-04,147,130,...,645,30,30,129,129,1,20170204,20170303,0,4.300000
zXjQ9f4C/uJHWw5guWICNbkFBydqdrz77Apch1nyDQ4=,1,0,NaN,13,20170108,0,2017-02-28,2017-01-09,50,16,...,258,30,30,129,129,1,20170208,20170307,0,4.300000


In [20]:
#Write all features to pkl
features_all.to_pickle('features_all.pkl')